In [1]:
%pip install openmeteo-requests
%pip install requests-cache retry-requests numpy pandas

   ---------------------------------------- 0.0/683.2 kB ? eta -:--:--
   ---------------------------------------- 0.0/683.2 kB ? eta -:--:--
   ---------------------------------------- 0.0/683.2 kB ? eta -:--:--
   --------------- ------------------------ 262.1/683.2 kB ? eta -:--:--
   --------------- ------------------------ 262.1/683.2 kB ? eta -:--:--
   ----------------------------- -------- 524.3/683.2 kB 762.0 kB/s eta 0:00:01
   ---------------------------------------- 683.2/683.2 kB 769.7 kB/s  0:00:01
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.5/2.0 MB 1.0 MB/s eta 0:00:02
   --------------- ------------------------ 0.8/2.0 MB 1.1 MB/s eta 0:00:02
   --------------------- ------------------ 1.0/2.0 MB 1.1 MB/s eta 0:00:01
   -------------------------- ------------- 1.3

In [4]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": -7.0520702239386175,
	"longitude": 110.43532807750137,
	"start_date": "2000-01-01",
	"end_date": "2005-12-31",
	"daily": ["temperature_2m_max", "temperature_2m_min", "weather_code", "temperature_2m_mean"],
	"hourly": ["temperature_2m", "weather_code", "relative_humidity_2m", "pressure_msl", "wind_speed_10m", "rain", "precipitation", "apparent_temperature", "surface_pressure"],
	"timezone": "Asia/Bangkok",
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone: {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_weather_code = hourly.Variables(1).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(2).ValuesAsNumpy()
hourly_pressure_msl = hourly.Variables(3).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(4).ValuesAsNumpy()
hourly_rain = hourly.Variables(5).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(6).ValuesAsNumpy()
hourly_apparent_temperature = hourly.Variables(7).ValuesAsNumpy()
hourly_surface_pressure = hourly.Variables(8).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end =  pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["weather_code"] = hourly_weather_code
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["pressure_msl"] = hourly_pressure_msl
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["rain"] = hourly_rain
hourly_data["precipitation"] = hourly_precipitation
hourly_data["apparent_temperature"] = hourly_apparent_temperature
hourly_data["surface_pressure"] = hourly_surface_pressure

hourly_dataframe = pd.DataFrame(data = hourly_data)
print("\nHourly data\n", hourly_dataframe)

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
daily_weather_code = daily.Variables(2).ValuesAsNumpy()
daily_temperature_2m_mean = daily.Variables(3).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end =  pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}

daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["weather_code"] = daily_weather_code
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean

daily_dataframe = pd.DataFrame(data = daily_data)
print("\nDaily data\n", daily_dataframe)


Coordinates: -7.06502628326416°N 110.4476318359375°E
Elevation: 208.0 m asl
Timezone: b'Asia/Bangkok'b'GMT+7'
Timezone difference to GMT+0: 25200s

Hourly data
                            date  temperature_2m  weather_code  \
0     1999-12-31 17:00:00+00:00       21.820999           3.0   
1     1999-12-31 18:00:00+00:00       21.370998           3.0   
2     1999-12-31 19:00:00+00:00       21.421000           3.0   
3     1999-12-31 20:00:00+00:00       21.171000           3.0   
4     1999-12-31 21:00:00+00:00       20.970999           3.0   
...                         ...             ...           ...   
52603 2005-12-31 12:00:00+00:00       23.021000          53.0   
52604 2005-12-31 13:00:00+00:00       22.671000          53.0   
52605 2005-12-31 14:00:00+00:00       22.021000          55.0   
52606 2005-12-31 15:00:00+00:00       21.620998          61.0   
52607 2005-12-31 16:00:00+00:00       21.771000          53.0   

       relative_humidity_2m  pressure_msl  wind_speed_10m 